In [3]:
from ics import Calendar, Event
c = Calendar()
e = Event()
e.name = "My cool event"
e.begin = '2021-02-01 00:00:00'
c.events.add(e)
c.events
# [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
with open('my.ics', 'w') as my_file:
    my_file.writelines(c)

## Converting things to UTC

In [39]:
from datetime import datetime
from dateutil import tz

In [40]:
dt_now = datetime.now(tz=tz.gettz('Canada/Pacific'))

In [44]:
dt_now.strftime("%Y-%m-%d %H:%M:%S")

'2020-08-23 23:01:55'

In [47]:
datetime.strptime('2020-08-23 23:01:55', '%Y-%m-%d %H:%M:%S')

datetime.datetime(2020, 8, 23, 23, 1, 55)

## Try the whole thing now

In [63]:
date_format = '%Y-%m-%d %H:%M:%S'
due = '2021-02-01 00:00:00'

due_dt_PDT = datetime.strptime(due,date_format).replace(tzinfo=tz.gettz('Canada/Pacific'))

print(due_dt_PDT)

2021-02-01 00:00:00-08:00


In [76]:
due_dt_UTC = due_dt_PDT.astimezone(tz.tzutc())
print(due_dt_UTC)

2021-02-01 08:00:00+00:00


In [67]:
from ics import Calendar, Event
c = Calendar()
e = Event()
e.name = "My cool event"
e.begin = due_dt_UTC.strftime(date_format)
c.events.add(e)
c.events
# [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
with open('my.ics', 'w') as my_file:
    my_file.writelines(c)

## Function to deal with time zones and datetimes

In [78]:
from datetime import datetime
from dateutil import tz

date_format = '%Y-%m-%d %H:%M:%S' # Format the ics module expects the dates to be in
due = '2021-02-01 00:00:00'       # Sample date that needs to be converted to UTC (Currently in Canada/Pacific time zone)

# First let's convert this to a datetime object in the Canada/Pacific Time zone
due_dt = datetime.strptime(due,date_format).replace(tzinfo=tz.gettz('Canada/Pacific'))

# Then let's convert the date to UTC so ics can handle it correctly
due_dt = due_dt.astimezone(tz.tzutc())

# Now let's run the example in the documentation
from ics import Calendar, Event
c = Calendar()
e = Event()
e.name = "My cool event"
e.begin = due_dt.strftime(date_format) # Updated this to use date_format
c.events.add(e)
c.events
# [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
with open('my.ics', 'w') as my_file:
    my_file.writelines(c)


In [75]:
due_dt

datetime.datetime(2021, 2, 1, 8, 0, tzinfo=tzutc())